### Generating operating data using simulation

We can use our model to generate a wide range of operation data to:
- learn about the product behavior
- anticipate how uncertainties will affect our understanding
- have data to feed AI algos
- etc.

In [ ]:
from cpu.systems import CPUSystem

# create a new CPU model/system
cpu = CPUSystem("cpu")

# load the characteristics
with open("data/cpu_ref.json") as f:
    cpu.load(f)

Now let's define a use case and apply some bias on the model and its data to represent real life operating data

In [ ]:
import numpy as np
import pandas as pd 

from cosapp.drivers import EulerExplicit, NonLinearSolver
from cosapp.drivers.time.scenario import Interpolator
from cosapp.recorders import DataFrameRecorder

tamb = np.random.uniform(39.5, 40.5, 31)
usage = np.concatenate([np.random.uniform(80., 100., 20), np.full(11, 0.)])

time_driver = cpu.add_driver(EulerExplicit())
solver = time_driver.add_child(NonLinearSolver('solver', max_iter=10, factor=1.0))

time_driver.time_interval = (0, 30)
time_driver.dt = 1.

time_driver.set_scenario(
    init = {'T_cpu': 10., "exchanger.h_adder": -30.},
    values = {
        "fan.T_air": Interpolator(np.stack([np.arange(0, 31, 1), tamb], axis=1)),
        "cpu.usage": Interpolator(np.stack([np.arange(0, 31, 1), usage], axis=1)),
    }
)

rec = time_driver.add_recorder(DataFrameRecorder(includes=['*'], hold=False), period=1.)

cpu.run_drivers()

data = rec.data[["fan.T_air", "cpu.usage", "T_cpu", "fan.tension", "time"]]
# T_cpu is here generated by simulation, add some noise/uncertainties
data["T_cpu"] += np.random.uniform(-0.2, 0.2, tamb.shape)

data.to_csv("data/cpu_hot_day_intensive_use.csv")

In [ ]:
from cpu.utils.plot import plot_recorders

plot_recorders({"theoretical": pd.read_csv("data/simulation_at_generic_conditions.csv"),
                "operation": pd.read_csv("data/cpu_hot_day_intensive_use.csv")
                }, 
               [[("time", "fan.T_air"), ("time","cpu.usage")], 
                [("time", "fan.tension"), ("time", "T_cpu")]],
               width=800, height=600)